# Analyse des Propriétés de l'État : Une Carte Choropleth pour Comprendre la Répartition Départementale


L'analyse présentée dans ce notebook se concentre sur les données relatives aux biens immobiliers appartenant à l'État français, issues du dataset "Inventaire Immobilier de l'État" disponible sur le site data.economie.gouv.fr. Ce dataset fournit des informations détaillées sur la localisation, le type, la fonction et le ministère dont dépendent ces biens immobiliers. L'objectif principal est de préparer les données pour créer une carte choropleth qui illustre la répartition des propriétés de l'État au niveau départemental.


L'utilisation de ces données permet non seulement de comprendre la répartition géographique des biens immobiliers de l'État mais aussi d'analyser leur diversité en termes de type, de fonction et de dépendance ministérielle. La source des données, https://data.economie.gouv.fr/api/explore/v2.1/catalog/datasets/inventaire-immobilier-de-letat/exports/csv?use_labels=true, offre un aperçu exhaustif de l'inventaire immobilier de l'État, permettant ainsi des analyses approfondies sur la base de critères variés.


## Méthodologie


La méthodologie adoptée pour cette analyse implique plusieurs étapes clés. Premièrement, les données sont extraites du dataset "Inventaire Immobilier de l'État" et chargées dans un dataframe pour traitement. Ensuite, une requête SQL est exécutée via DuckDB pour agréger les données par département, en calculant le nombre total de propriétés, le nombre de propriétés uniques et le nombre de ministères différents représentés dans chaque département. Les résultats sont ensuite utilisés pour créer une carte choropleth à l'aide de Plotly Express, en superposant les données agrégées avec un GeoJSON représentant les départements français.


Les données sont ainsi visualisées de manière à mettre en évidence les disparités régionales dans la répartition des propriétés de l'État, facilitant l'identification des départements avec une forte concentration de biens immobiliers et permettant une analyse plus fine de la diversité de ces biens en fonction de différents critères.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.economie.gouv.fr/api/explore/v2.1/catalog/datasets/inventaire-immobilier-de-letat/exports/csv?use_labels=true", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
  lpad("dept", 2, '0') AS code_dept,
  COUNT(*) AS nb_proprietes,
  COUNT(DISTINCT "id") AS nb_proprietes_uniques,
  COUNT(DISTINCT "ministere") AS nb_ministeres
FROM 
  loaded_dataset
GROUP BY 
  "dept"
ORDER BY 
  code_dept """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée pour cette datavisualisation est Plotly Express, qui est idéale pour créer des cartes interactives et des représentations graphiques complexes de manière simple et efficace. Le choix d'une carte choropleth est pertinent pour représenter des données géographiques, telles que le nombre de propriétés de l'État par département en France. Cela permet une visualisation claire et intuitive des données.

In [3]:
import pandas as pd
import duckdb as ddb
import plotly.express as px
import pandas as pd
import requests
import json

# Récupération du geojson des départements français
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
response = requests.get(url)
geojson = response.json()

# Mise en forme du code_dept en 2 chiffres
df['code_dept'] = df['code_dept'].astype(str).str.zfill(2)

# Création de la carte choropleth
dataviz = px.choropleth_mapbox(
    df,
    geojson=geojson,
    locations='code_dept',
    featureidkey="properties.code",
    color='nb_proprietes',
    color_continuous_scale="Blues",
    range_color=[df['nb_proprietes'].min(), df['nb_proprietes'].max()],
    mapbox_style="carto-positron",
    zoom=4.5,
    center={"lat": 46.5, "lon": 2.5},
    opacity=0.7,
    labels={'nb_proprietes': "Nombre de propriétés"},
    hover_data={
        'code_dept': True,
        'nb_proprietes': True,
        'nb_proprietes_uniques': True,
        'nb_ministeres': True
    }
)

# Mise en page finale
dataviz.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    title={
        'text': "Carte des propriétés de l'État par département",
        'x': 0.5,
        'xanchor': 'center',
        'font': dict(size=16, family="Segoe UI")
    },
    coloraxis_colorbar_title_text="Nb propriétés",
    coloraxis_colorbar_len=0.8
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*